In [1]:
import pandas as pd
import numpy as np
from keras import models, layers, regularizers
from matplotlib import pyplot as plt

Using TensorFlow backend.


# Load Data

In [2]:
data = pd.read_csv("CarPrice_Assignment.csv")
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [3]:
data.shape

(205, 26)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
car_ID              205 non-null int64
symboling           205 non-null int64
CarName             205 non-null object
fueltype            205 non-null object
aspiration          205 non-null object
doornumber          205 non-null object
carbody             205 non-null object
drivewheel          205 non-null object
enginelocation      205 non-null object
wheelbase           205 non-null float64
carlength           205 non-null float64
carwidth            205 non-null float64
carheight           205 non-null float64
curbweight          205 non-null int64
enginetype          205 non-null object
cylindernumber      205 non-null object
enginesize          205 non-null int64
fuelsystem          205 non-null object
boreratio           205 non-null float64
stroke              205 non-null float64
compressionratio    205 non-null float64
horsepower          205 non-null int64
peakrpm      

In [5]:
data.describe()

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,103.000000,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,59.322565,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,1.000000,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,52.000000,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,103.000000,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,154.000000,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,205.000000,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


In [6]:
#Separating the company name and car name
companyName = data["CarName"].apply(lambda x: x.split(" ")[0])
data.insert(3,"companyname",companyName)
data.head()

,car_ID,symboling,CarName,companyname,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,alfa-romero,gas,std,two,convertible,rwd,front,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,alfa-romero,gas,std,two,convertible,rwd,front,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,alfa-romero,gas,std,two,hatchback,rwd,front,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,audi,gas,std,four,sedan,fwd,front,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,audi,gas,std,four,sedan,4wd,front,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [7]:
#Drop the garbage column of Carname
data.drop("CarName", axis=1, inplace=True)
data.head()

,car_ID,symboling,companyname,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [8]:
print(data.companyname.unique())
print(data.fueltype.unique())
print(data.aspiration.unique())
print(data.doornumber.unique())
print(data.carbody.unique())
print(data.drivewheel.unique())
print(data.enginelocation.unique())
print(data.enginetype.unique())
print(data.cylindernumber.unique())
print(data.fuelsystem.unique())

['alfa-romero' 'audi' 'bmw' 'chevrolet' 'dodge' 'honda' 'isuzu' 'jaguar'
 'maxda' 'mazda' 'buick' 'mercury' 'mitsubishi' 'Nissan' 'nissan'
 'peugeot' 'plymouth' 'porsche' 'porcshce' 'renault' 'saab' 'subaru'
 'toyota' 'toyouta' 'vokswagen' 'volkswagen' 'vw' 'volvo']
['gas' 'diesel']
['std' 'turbo']
['two' 'four']
['convertible' 'hatchback' 'sedan' 'wagon' 'hardtop']
['rwd' 'fwd' '4wd']
['front' 'rear']
['dohc' 'ohcv' 'ohc' 'l' 'rotor' 'ohcf' 'dohcv']
['four' 'six' 'five' 'three' 'twelve' 'two' 'eight']
['mpfi' '2bbl' 'mfi' '1bbl' 'spfi' '4bbl' 'idi' 'spdi']


In [9]:
#setting the wrong spelling mistakes

#For Company Name
data.companyname = data.companyname.str.lower()
def correct_name(wrong,right):
    data.companyname.replace(wrong,right,inplace=True)
correct_name("maxda","mazda")
correct_name("porcshce", "porsche")
correct_name("toyouta","toyota")
correct_name("vokswagen", "volkswagen")
correct_name("vw","volkswagen")
data.companyname.unique()


#For Engine Location
data.drivewheel.replace("4wd","fwd",inplace=True)

In [10]:
data = pd.get_dummies(data, prefix=["companyname",'fueltype','aspiration', 'doornumber','carbody', 'drivewheel',
                                    'enginelocation', 'enginetype','cylindernumber', 'fuelsystem'])
data

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,...,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
0,1,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,...,0,0,0,0,0,0,0,1,0,0
1,2,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,...,0,0,0,0,0,0,0,1,0,0
2,3,1,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,...,0,0,0,0,0,0,0,1,0,0
3,4,2,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,...,0,0,0,0,0,0,0,1,0,0
4,5,2,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,109.1,188.8,68.9,55.5,2952,141,3.78,3.15,...,0,0,0,0,0,0,0,1,0,0
201,202,-1,109.1,188.8,68.8,55.5,3049,141,3.78,3.15,...,0,0,0,0,0,0,0,1,0,0
202,203,-1,109.1,188.8,68.9,55.5,3012,173,3.58,2.87,...,0,0,0,0,0,0,0,1,0,0
203,204,-1,109.1,188.8,68.9,55.5,3217,145,3.01,3.40,...,0,0,0,0,0,1,0,0,0,0


# Vectorize the data

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
data["fueltype"]=labelencoder.fit_transform(data["fueltype"])
data["aspiration"]=labelencoder.fit_transform(data["aspiration"])
data["carbody"]=labelencoder.fit_transform(data["carbody"])
data["drivewheel"]=labelencoder.fit_transform(data["drivewheel"])
data["enginetype"]=labelencoder.fit_transform(data["enginetype"])
data["cylindernumber"]=labelencoder.fit_transform(data["cylindernumber"])
data["enginelocation"]=labelencoder.fit_transform(data["enginelocation"])
data["companyname"]=labelencoder.fit_transform(data["companyname"])
data["doornumber"]=labelencoder.fit_transform(data["doornumber"])
data["fuelsystem"]=labelencoder.fit_transform(data["fuelsystem"])
data.head()

# Missing values

In [11]:
data.isnull().sum()

car_ID             0
symboling          0
wheelbase          0
carlength          0
carwidth           0
                  ..
fuelsystem_idi     0
fuelsystem_mfi     0
fuelsystem_mpfi    0
fuelsystem_spdi    0
fuelsystem_spfi    0
Length: 75, dtype: int64

In [12]:
data.isna().sum()

car_ID             0
symboling          0
wheelbase          0
carlength          0
carwidth           0
                  ..
fuelsystem_idi     0
fuelsystem_mfi     0
fuelsystem_mpfi    0
fuelsystem_spdi    0
fuelsystem_spfi    0
Length: 75, dtype: int64

# Feature Engineering

In [13]:
data_new = data.copy()
fueleconomy = (0.55 * data_new['citympg']) + (0.45 * data_new['highwaympg'])
carvolume = data_new['carlength'] * data_new['carwidth'] * data_new['carheight']
data_new.insert(3,"fueleconomy",fueleconomy)
data_new.insert(3,"carvolume",carvolume)
data_new.drop("carlength", axis=1, inplace=True)
data_new.drop("carwidth", axis=1, inplace=True)
data_new.drop("carheight", axis=1, inplace=True)
data_new.drop("citympg", axis=1, inplace=True)
data_new.drop("highwaympg", axis=1, inplace=True)
#data_new.drop("curbweight", axis=1, inplace=True)
#data_new.drop("cylindernumber", axis=1, inplace=True)
#data_new.drop("peakrpm", axis=1, inplace=True)
#data_new.drop("doornumber", axis=1, inplace=True)
#data_new.drop("enginetype", axis=1, inplace=True)
#data_new.drop("compressionratio", axis=1, inplace=True)
#data_new.drop("enginelocation", axis=1, inplace=True)
data_new.head()

,car_ID,symboling,wheelbase,carvolume,fueleconomy,curbweight,enginesize,boreratio,stroke,compressionratio,...,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
0,1,3,88.6,528019.904,23.70,2548,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,1,0,0
1,2,3,88.6,528019.904,23.70,2548,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,1,0,0
2,3,1,94.5,587592.640,22.15,2823,152,2.68,3.47,9.0,...,0,0,0,0,0,0,0,1,0,0
3,4,2,99.8,634816.956,26.70,2337,109,3.19,3.40,10.0,...,0,0,0,0,0,0,0,1,0,0
4,5,2,99.4,636734.832,19.80,2824,136,3.19,3.40,8.0,...,0,0,0,0,0,0,0,1,0,0


# Splitting data into training, test

In [ ]:
np.random.seed(111)
part = np.random.randn(len(data_new)) < 0.7
train_data = data_new[part]
test_data = data_new[~part]
train_target = train_data.pop("price")
test_target = test_data.pop("price")

# Normalize the data

In [ ]:
data_mean = train_data.mean(axis=0)
train_data -= data_mean   
data_std = train_data.std(axis=0)
train_data /= data_std
test_data -= data_mean
test_data/= data_std

# Driving validation set from training set

In [ ]:
train_data = train_data[:103]
val_train_data = train_data[103:]
train_target = train_target[:103]
val_train_target = train_target[103:]

In [ ]:
# Making the Model
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(10,activation = "relu", input_shape= (train_data.shape[1],)))
    model.add(layers.Dense(8,activation = "relu"))
    model.add(layers.Dense(6, activation = "relu"))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [ ]:
model = build_model()
model.fit(train_data,
train_target,
epochs=100,
batch_size=128)

In [ ]:
#k fold validation and saving validation logs at each epocs
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_target[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = np.concatenate([train_target[:i * num_val_samples],train_target[(i + 1) * num_val_samples:]],axis=0)
model = build_model()
history = model.fit(partial_train_data, partial_train_targets,
validation_data=(val_data, val_targets),epochs=num_epochs, batch_size=1, verbose=0)
mae_history = history.history['val_mae']
all_mae_histories.append(mae_history)
history.history

In [ ]:
#building the history ofsuccessive mean k_fold validation scores
average_mae_history = [
np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
#ploting while removing first 10 data points
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points
smooth_mae_history = smooth_curve(average_mae_history[90:])
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
model = build_model()
model.fit(train_data, train_target,
epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_target)

In [ ]:
test_mse_score

# Split train and test set

In [14]:
y = data["price"]
y

0      13495.0
1      16500.0
2      16500.0
3      13950.0
4      17450.0
        ...   
200    16845.0
201    19045.0
202    21485.0
203    22470.0
204    22625.0
Name: price, Length: 205, dtype: float64

In [ ]:
data.drop("price",axis = 1, inplace=True)

In [ ]:
X_train = data[:104]
Y_train = y[:104]
X_test = data[104:165]
Y_test = y[104:165]
X_val = data[165:]
Y_val = y[165:]

In [ ]:
data_mean = X_train.mean(axis=0)
X_train -= data_mean   
data_std = Y_train.std(axis=0)
Y_train /= data_std
X_test -= data_mean
Y_test/= data_std

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', kernel_regularizer =regularizers.l2(0.02), input_shape=(X_train.shape[1],)))
model.add(layers.Dense(14, activation='relu', kernel_regularizer =regularizers.l2(0.02)))
model.add(layers.Dense(8, activation='relu', kernel_regularizer =regularizers.l2(0.02)))
model.add(layers.Dense(6, activation='relu', kernel_regularizer =regularizers.l2(0.002)))
# network.add(layers.Dense(4, activation='relu', kernel_regularizer =regularizers.l2(0.002)))
model.add(layers.Dense(1))

In [ ]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [ ]:
history = model.fit(X_train,Y_train, batch_size=16, verbose=0, epochs=200, validation_data=(X_val, Y_val))
history

In [ ]:
epochs = range(1, 201)
train_mae = history.history['mae']
val_mae = history.history['val_mae']

In [ ]:
plt.plot(epochs, train_mae, 'r', label='Training MAE')
plt.plot(epochs, val_mae, 'g', label='Validation MAE')
plt.title('Training and Validation loss ')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()